In [5]:
import os.path as osp

import torch
from torch.utils.data import DataLoader
from torch_geometric.datasets import Planetoid
from torch_geometric_node2vec import Node2Vec

dataset = 'Cora'
dataset = Planetoid('/tmp/n2vtest', dataset)
data = dataset[0]
loader = DataLoader(torch.arange(data.num_nodes), batch_size=128, shuffle=True)

device = 'cpu'
model = Node2Vec(data.num_nodes, embedding_dim=128, walk_length=20,
                 context_size=10, walks_per_node=10)
model, data = model.to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train():
    model.train()
    total_loss = 0
    for subset in loader:
        optimizer.zero_grad()
        loss = model.loss(data.edge_index, subset.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


def test():
    model.eval()
    z = model(torch.arange(data.num_nodes, device=device))
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask], max_iter=150)
    return acc


for epoch in range(1, 101):
    loss = train()
    print('Epoch: {:02d}, Loss: {:.4f}'.format(epoch, loss))
acc = test()
print('Accuracy: {:.4f}'.format(acc))

Epoch: 01, Loss: 8.0818
Epoch: 02, Loss: 6.0601
Epoch: 03, Loss: 4.9704
Epoch: 04, Loss: 4.1561
Epoch: 05, Loss: 3.5119
Epoch: 06, Loss: 3.0022
Epoch: 07, Loss: 2.5892
Epoch: 08, Loss: 2.2635
Epoch: 09, Loss: 2.0013
Epoch: 10, Loss: 1.7883
Epoch: 11, Loss: 1.6192
Epoch: 12, Loss: 1.4800
Epoch: 13, Loss: 1.3672
Epoch: 14, Loss: 1.2765
Epoch: 15, Loss: 1.2033
Epoch: 16, Loss: 1.1429
Epoch: 17, Loss: 1.0958
Epoch: 18, Loss: 1.0570
Epoch: 19, Loss: 1.0247
Epoch: 20, Loss: 1.0010
Epoch: 21, Loss: 0.9788
Epoch: 22, Loss: 0.9621
Epoch: 23, Loss: 0.9472
Epoch: 24, Loss: 0.9331
Epoch: 25, Loss: 0.9235
Epoch: 26, Loss: 0.9135
Epoch: 27, Loss: 0.9074
Epoch: 28, Loss: 0.8986
Epoch: 29, Loss: 0.8932
Epoch: 30, Loss: 0.8893
Epoch: 31, Loss: 0.8847
Epoch: 32, Loss: 0.8794
Epoch: 33, Loss: 0.8773
Epoch: 34, Loss: 0.8741
Epoch: 35, Loss: 0.8707
Epoch: 36, Loss: 0.8694
Epoch: 37, Loss: 0.8671
Epoch: 38, Loss: 0.8640
Epoch: 39, Loss: 0.8618
Epoch: 40, Loss: 0.8606
Epoch: 41, Loss: 0.8604
Epoch: 42, Loss:

In [10]:
model(torch.tensor([1,2])).shape

torch.Size([2, 128])

In [11]:
data.x[[1,2]].shape

torch.Size([2, 1433])

In [17]:
c = torch.cat((model(torch.tensor([1,2])),data.x[[1,2]]),dim=1)

In [18]:
c.shape

torch.Size([2, 1561])

In [26]:
torch.nn.functional.softmax(c,dim=1)

tensor([[0.0010, 0.0006, 0.0006,  ..., 0.0006, 0.0006, 0.0006],
        [0.0009, 0.0006, 0.0007,  ..., 0.0006, 0.0006, 0.0006]],
       grad_fn=<SoftmaxBackward>)